## Make predictions

In [7]:
import os 
import sys
import time 
import json 
import torch 
import random 
import warnings
import torchvision
import numpy as np 
import pandas as pd

from utils import *
from data import PlanetDataset
from tqdm import tqdm
from config_weightedce import config
from datetime import datetime
from models.model import *
from torch import nn,optim
from collections import OrderedDict
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from sklearn.model_selection import train_test_split, KFold
from timeit import default_timer as timer

%load_ext autoreload
%autoreload 2

In [18]:
# test model on public dataset and save the probability matrix
def test(test_loader, model, folds, all_train_loader=None):
#     sample_submission_df = pd.read_csv("./dataset/test_visit.csv")
    #3.1 confirm the model converted to cuda
    filenames, labels_te, labels_tr, submissions= [],[],[],[]
    labels_true = []
    model.cuda()
    model.eval()
    submit_results = []
    try:
        if not isinstance(test_loader, type(None)):
            with tqdm(test_loader, ascii = True) as t:
                for img_data, visit_data, filepath in t:
                    #3.2 change everything to cuda and get only basename
                    filepath = [x for x in filepath]
                    with torch.no_grad():
                        image_var = img_data.cuda(non_blocking=True)
                        visit_var = visit_data.cuda(non_blocking=True)
                        y_pred = model(image_var, visit_var)
                        label = y_pred.cpu().data.numpy()
                        #print(label > 0.5)

                        submit_results.append(np.argmax(label, 1))
                        labels_te.append(np.squeeze(label))

                        filenames.append(np.squeeze(filepath))

        if not isinstance(all_train_loader, type(None)):
            with tqdm(all_train_loader, ascii = True) as t:
                for img_data, visit_data, target in t:
                    #3.2 change everything to cuda and get only basename
                    target = np.squeeze(target).cpu().data.numpy()
                    with torch.no_grad():
                        image_var = img_data.cuda(non_blocking=True)
                        visit_var = visit_data.cuda(non_blocking=True)
                        y_pred = model(image_var, visit_var)
                        label = y_pred.cpu().data.numpy()
                        labels_tr.append(np.hstack([np.squeeze(label), target.reshape(-1, 1)]))
            return None, np.concatenate(labels_tr)#np.concatenate(labels_te)
    except:
        t.close()
        raise
    t.close()

# 	sample_submission_df['Predicted'] = np.concatenate(submit_results)
# 	sample_submission_df['IterId'] = np.concatenate(filenames)
# 	sample_submission_df.to_csv('./submit/%s_bestloss_submission%s.csv'%(config.model_name, folds), index=None)
    return np.concatenate(labels_te), None

In [11]:
# test model on public dataset and save the probability matrix
def test(test_loader, model, folds, all_train_loaders=None):
#     sample_submission_df = pd.read_csv("./dataset/test_visit.csv")
    #3.1 confirm the model converted to cuda
    [all_train_loader, b_train_loader] = all_train_loaders
    
    filenames, labels_te, labels_tr, labels_b, submissions= [],[],[],[], []
    labels_true = []
    model.cuda()
    model.eval()
    submit_results = []
    try:
        if not isinstance(test_loader, type(None)):
            with tqdm(test_loader, ascii = True) as t:
                for img_data, visit_data, filepath in t:
                    #3.2 change everything to cuda and get only basename
                    filepath = [x for x in filepath]
                    with torch.no_grad():
                        image_var = img_data.cuda(non_blocking=True)
                        visit_var = visit_data.cuda(non_blocking=True)
                        y_pred = model(image_var, visit_var)
                        label = y_pred.cpu().data.numpy()
                        #print(label > 0.5)

                        submit_results.append(np.argmax(label, 1))
                        labels_te.append(np.squeeze(label))

                        filenames.append(np.squeeze(filepath))

        if not isinstance(all_train_loader, type(None)):
            with tqdm(all_train_loader, ascii = True) as t:
                for img_data, visit_data, target in t:
                    #3.2 change everything to cuda and get only basename
#                     return target
                    target = np.squeeze(target).cpu().data.numpy()
                    with torch.no_grad():
                        image_var = img_data.cuda(non_blocking=True)
                        visit_var = visit_data.cuda(non_blocking=True)
                        y_pred = model(image_var, visit_var)
                        label = y_pred.cpu().data.numpy()
                        labels_tr.append(np.hstack([np.squeeze(label), target.reshape(-1, 1)]))
            with tqdm(b_train_loader, ascii = True) as t:
                for img_data, visit_data, target in t:
                    #3.2 change everything to cuda and get only basename
#                     return target
                    target = np.squeeze(target).cpu().data.numpy()
                    with torch.no_grad():
                        image_var = img_data.cuda(non_blocking=True)
                        visit_var = visit_data.cuda(non_blocking=True)
                        y_pred = model(image_var, visit_var)
                        label = y_pred.cpu().data.numpy()
                        labels_b.append(np.hstack([np.squeeze(label), target.reshape(-1, 1)]))
            return np.concatenate(labels_te), np.concatenate(labels_tr), np.concatenate(labels_b)#np.concatenate(labels_te)
    except:
        t.close()
        raise
    t.close()

# 	sample_submission_df['Predicted'] = np.concatenate(submit_results)
# 	sample_submission_df['IterId'] = np.concatenate(filenames)
# 	sample_submission_df.to_csv('./submit/%s_bestloss_submission%s.csv'%(config.model_name, folds), index=None)
    return np.concatenate(labels_te), None

In [8]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0, 1, 2, 3"
torch.backends.cudnn.benchmark = True

warnings.filterwarnings('ignore')

In [5]:
# config.model_name = "seresnext5032x4d_resnet110-FLlog-03washing-vp-adjustlr-v0pretrained"  # 7*24*26visit 的模型名称
# config.model_name = "seresnext5032x4d_resnet110-celog-03washing-182vp-adjustlr-skf" # 24*182visit 的模型名称
config.model_name = "seresnext5032x4d_resnet110-CE-03washing-v0-segpre"
config.model_name = "seresnext5032x4d_resnet110-CE-03washing-v0-imgpre"#addcs

In [9]:
print('# discriminator parameters:', sum(param.numel() for param in model.parameters()))

# discriminator parameters: 27502793


In [4]:
config.model_name = "seresnext5032x4d-FLlog-03washing-lr28-v0-final-batch256"

In [4]:
config.model_name = "resnet110-visituser-CElog-03washing-lr26-v0-final-nonorm"

In [10]:
model

DataParallel(
  (module): MultiModalNet(
    (image_model): SENet(
      (layer0): Sequential(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (layer1): Sequential(
        (0): SEResNeXtBottleneck(
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=

In [12]:
if __name__ == "__main__":
    all_files = pd.read_csv("./dataset/final_train.csv")
#     all_files.Id_y = all_files.Id_y.apply(lambda x: x.replace("visit", "visit182"))
    all_files = all_files.loc[~(all_files.black_ratio<0.3)]#|(all_files.same_pixel>70)
	#print(all_files)
    test_files = pd.read_csv("./dataset/final_test.csv")
    # test_files.Id_x = test_files.Id_x.apply(lambda x: "."+x)
#     test_files.Id_y = test_files.Id_y.apply(lambda x: x.replace("visit", "visit182"))

#     config.model_name = "seresnext5032x4d_resnet110-CE-03washing-v0-182v-imgpre-addchusai"#"seresnext5032x4d_resnet110-CE-03washing-v0-segpre"
#     config.model_name = "seresnext5032x4d_resnet110-CE-03washing-v0-182v+log-segpre-addchusaino0-imgdehaze"#去噪+dist+logv
    all_pred = []
    all_pred_tr = []
    all_pred_val = []
    labels_tr = []
    labels_val = []
    for fold in range(5):
        print( "\nFold ", fold)
        model = MultiModalNet(config, sep_pretrained=True, visit_channels=7, fold=fold)
        model.modify_model()
        val_files = pd.read_csv("./temp/final_X_val%s.csv"%fold)
        # MultiGpu
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        if torch.cuda.device_count() >= 1:
            print("Let's use", torch.cuda.device_count(), "GPUs")
            # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
            model = nn.DataParallel(model)#, device_ids=[0, 2]
        model.to(device)

        best_model = torch.load("%s/%s_fold_%s_model_best_loss.pth.tar"%(config.best_models, config.model_name, str(fold)))
        model.load_state_dict(best_model["state_dict"])
    
        for i in range(8):
            all_train_gen = PlanetDataset(val_files, augument=False, mode="test", dtype="mixed", config=config, transforms_id=i)
            all_train_loader = DataLoader(all_train_gen, 256*4,shuffle=False,pin_memory=True,num_workers=16)
            b_train_gen = PlanetDataset(all_files, augument=False, mode="test", dtype="mixed", config=config, transforms_id=i)
            b_train_loader = DataLoader(b_train_gen, 256*4,shuffle=False,pin_memory=True,num_workers=16)
            test_gen = PlanetDataset(test_files, augument=False, mode="test", dtype="mixed", config=config, transforms_id=i)
            test_loader = DataLoader(test_gen, 256*4, shuffle=False, pin_memory=True, num_workers=16)

            labels_fold, labels_tr_fold, labels_b_fold = test(test_loader, model, fold, all_train_loaders=[all_train_loader, b_train_loader])
            all_pred.append(labels_fold)
            all_pred_tr.append(labels_tr_fold)
            all_pred_val.append(labels_b_fold)
#             all_pred_tr.append(labels_tr_fold)
#         np.save("./results/test182_addcs_bestloss.npy", all_pred)
#         np.save("./results/train182_addcs_bestloss.npy", all_pred)
torch.cuda.empty_cache()


Fold  0
train seresnext5032x4d
train resnet110
Single model pretrained respectively
=> loading checkpoint './checkpoints/best_models//seresnext5032x4d-img224-FLlog-03washing-lr28-v0-final-batch32_fold_0_model_best_loss.pth.tar'
Modifing model...
Let's use 4 GPUs


100%|####################################################################################| 2/2 [00:53<00:00, 37.27s/it]



Fold  1
train seresnext5032x4d
train resnet110
Single model pretrained respectively
=> loading checkpoint './checkpoints/best_models//seresnext5032x4d-img224-FLlog-03washing-lr28-v0-final-batch32_fold_1_model_best_loss.pth.tar'
Modifing model...
Let's use 4 GPUs


100%|####################################################################################| 2/2 [00:53<00:00, 37.17s/it]



Fold  2
train seresnext5032x4d
train resnet110
Single model pretrained respectively
=> loading checkpoint './checkpoints/best_models//seresnext5032x4d-img224-FLlog-03washing-lr28-v0-final-batch32_fold_2_model_best_loss.pth.tar'
Modifing model...
Let's use 4 GPUs


100%|####################################################################################| 2/2 [00:53<00:00, 37.04s/it]



Fold  3
train seresnext5032x4d
train resnet110
Single model pretrained respectively
=> loading checkpoint './checkpoints/best_models//seresnext5032x4d-img224-FLlog-03washing-lr28-v0-final-batch32_fold_3_model_best_loss.pth.tar'
Modifing model...
Let's use 4 GPUs


100%|####################################################################################| 2/2 [00:53<00:00, 37.20s/it]



Fold  4
train seresnext5032x4d
train resnet110
Single model pretrained respectively
=> loading checkpoint './checkpoints/best_models//seresnext5032x4d-img224-FLlog-03washing-lr28-v0-final-batch32_fold_4_model_best_loss.pth.tar'
Modifing model...
Let's use 4 GPUs


100%|####################################################################################| 2/2 [00:53<00:00, 37.21s/it]


In [13]:
results = []
for i in range(5):
    results.append(np.mean(all_pred_tr[i*8: (i*8+8)], 0))
results_tr = np.vstack(results)
print(results_tr.shape)

(398659, 10)


In [14]:
results_tr[:, 9] = results_tr[:, 9].astype("int")

In [15]:
results_tr[:, 9]

array([1.30000e+01, 1.70000e+01, 2.10000e+01, ..., 3.99881e+05,
       3.99896e+05, 3.99988e+05])

In [16]:
results = []
for i in range(5):
    results.append(np.mean(all_pred[i*8: (i*8+8)], 0))
results_te = np.mean(results, 0)
results_te.shape

(100000, 9)

In [17]:
results = []
for i in range(5):
    results.append(np.mean(all_pred_val[i*8: (i*8+8)], 0))
results_tr_b = np.mean(results, 0)
results_tr_b.shape

(1341, 10)

In [17]:
results_tr_b = np.hstack((results_tr_b, test_files.basename.values.reshape((-1, 1))))

In [18]:
results = np.vstack([results_tr, results_tr_b])
results.shape

(400000, 10)

In [17]:
np.save("./stacking/test7_oof_addcs_orig.npy", results_te)

In [19]:
np.save("./stacking/test7_oof_visit_orig.npy", np.mean(all_pred, 0))

In [21]:
np.save("./stacking/test7_192img_oof_orig.npy", results_te)

In [16]:
np.save("./stacking/train7_oof_visit_orig.npy", results)

In [19]:
np.save("./stacking/train7_192img_oof_orig.npy", results)

In [6]:
# visit
results = np.vstack([np.vstack(all_pred_tr), np.mean(all_pred_val, 0)])
print(results.shape)
np.save("./stacking/train7_oof_visit_orig.npy", results)
print(np.mean(all_pred, 0).shape)
np.save("./stacking/test7_oof_visit_orig.npy", np.mean(all_pred, 0))

(400000, 10)
(100000, 9)


In [29]:
# correct solution:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

In [33]:
softmax(results_tr_b.T).T[0]

array([0.5264755 , 0.06973959, 0.0022421 , 0.00553492, 0.00002445,
       0.02236212, 0.01376571, 0.02047716, 0.33937848], dtype=float32)

In [15]:
results_tr_b

array([[ 2.8940768 ,  0.8726406 , -2.5647166 , ..., -0.7499459 ,
        -0.3528169 ,  2.454988  ],
       [ 1.6143271 ,  0.1245259 ,  2.239495  , ...,  1.2305094 ,
        -1.6487957 , -1.5342503 ],
       [ 1.3956559 ,  0.92694914,  1.5661005 , ...,  1.0466396 ,
        -2.3046355 , -2.0994678 ],
       ...,
       [ 4.5450287 ,  0.77550787, -0.41005713, ..., -1.9173464 ,
        -0.7988874 , -0.8545507 ],
       [ 4.1012583 ,  1.5728486 , -0.5812508 , ...,  0.04408486,
        -0.93905944, -1.0305606 ],
       [ 4.200846  ,  0.21507902, -1.9294401 , ...,  1.3776729 ,
        -0.58340347, -0.29319698]], dtype=float32)

In [14]:
np.save("./stacking/test7_oof.npy", results_tr_b)

## Generate Submission

In [2]:
import pickle

In [18]:
np.set_printoptions(suppress=True)

In [18]:
fold5182 = np.squeeze(all_pred).copy()

In [11]:
fold5 = np.squeeze(all_pred).copy()
fold5.shape

(40, 100000, 9)

In [12]:
fold5 = np.mean(fold5, 0)
fold5.shape

(100000, 9)

In [19]:
data = np.zeros((5, 100000, 9))

for i in range(5):
    data[i, :, :] = np.mean(fold5[i:(i*8+8)], 0)

In [7]:
fold = np.load("./fusion/test7.npy")

In [9]:
fold5 = np.mean(fold, 0)
fold5.shape

(100000, 9)

In [3]:
with open("./fusion/meow_test1.pkl", 'rb') as f:
    model1 = pickle.load(f)
result = 0
for key in [str(i) for i in range(5)]:
    result += model1[key]
result /= 5

In [5]:
result.shape

(100000, 9)

In [11]:
result[:5]

array([[0.7964071 , 0.04848755, 0.00019563, 0.00148878, 0.00000043,
        0.03733421, 0.03366304, 0.01041587, 0.07200748],
       [0.04991202, 0.02756109, 0.12903196, 0.69894725, 0.00000005,
        0.01240712, 0.07457936, 0.00288484, 0.00467634],
       [0.32507944, 0.04853205, 0.36576387, 0.00094881, 0.10286842,
        0.08923764, 0.05260446, 0.00234388, 0.01262144],
       [0.593215  , 0.0276758 , 0.00049242, 0.00007384, 0.00000003,
        0.37284526, 0.00135877, 0.00196207, 0.00237675],
       [0.63252026, 0.09085535, 0.03664375, 0.00011   , 0.00000002,
        0.04316987, 0.05433962, 0.00964581, 0.13271542]], dtype=float32)

In [19]:
all_files = pd.read_csv("./dataset/train.csv")
all_files.basename = all_files.basename.apply(lambda x: int(x[:6])+400000)
all_files

In [22]:
pd.read_csv("./dataset/final_train.csv")

,Id_x,Target,black_ratio,basename,Id_y
0,./dataset/final_train/001\000003_001.jpg,0,0.000000,3,./dataset/train_partarray/3\000003_001.npy
1,./dataset/final_train/001\000007_001.jpg,0,0.000000,7,./dataset/train_partarray/7\000007_001.npy
2,./dataset/final_train/001\000008_001.jpg,0,0.000000,8,./dataset/train_partarray/8\000008_001.npy
3,./dataset/final_train/001\000009_001.jpg,0,0.000000,9,./dataset/train_partarray/9\000009_001.npy
4,./dataset/final_train/001\000013_001.jpg,0,0.000000,13,./dataset/train_partarray/3\000013_001.npy
5,./dataset/final_train/001\000015_001.jpg,0,0.000000,15,./dataset/train_partarray/5\000015_001.npy
6,./dataset/final_train/001\000016_001.jpg,0,0.000000,16,./dataset/train_partarray/6\000016_001.npy
7,./dataset/final_train/001\000017_001.jpg,0,0.000000,17,./dataset/train_partarray/7\000017_001.npy
8,./dataset/final_train/001\000021_001.jpg,0,0.000000,21,./dataset/train_partarray/1\000021_001.npy
9,./dataset/final_train/001\000024_001.jpg,0,0.000000,24,./dataset/train_partarray/4\000024_001.npy


In [21]:
pd.read_csv("./dataset/train.csv")

,Id_x,Target,black_ratio,basename,Id_y,max_min_pixel,same_pixel
0,./dataset/train\001\000009_001.jpg,0,0.000000,000009_001,./dataset/train_visitarray/000009_001.npy,"[765, 120]",645
1,./dataset/train\001\000018_001.jpg,0,0.000000,000018_001,./dataset/train_visitarray/000018_001.npy,"[760, 162]",598
2,./dataset/train\001\000020_001.jpg,0,0.000000,000020_001,./dataset/train_visitarray/000020_001.npy,"[763, 202]",561
3,./dataset/train\001\000024_001.jpg,0,0.000000,000024_001,./dataset/train_visitarray/000024_001.npy,"[763, 386]",377
4,./dataset/train\001\000026_001.jpg,0,0.000000,000026_001,./dataset/train_visitarray/000026_001.npy,"[763, 326]",437
5,./dataset/train\001\000029_001.jpg,0,0.000000,000029_001,./dataset/train_visitarray/000029_001.npy,"[764, 295]",469
6,./dataset/train\001\000031_001.jpg,0,0.000000,000031_001,./dataset/train_visitarray/000031_001.npy,"[755, 373]",382
7,./dataset/train\001\000037_001.jpg,0,0.000000,000037_001,./dataset/train_visitarray/000037_001.npy,"[757, 196]",561
8,./dataset/train\001\000062_001.jpg,0,0.000000,000062_001,./dataset/train_visitarray/000062_001.npy,"[643, 219]",424
9,./dataset/train\001\000063_001.jpg,0,0.000000,000063_001,./dataset/train_visitarray/000063_001.npy,"[752, 215]",537


In [ ]:
## data_old---->train
all_files_pre = pd.read_csv("./dataset/train.csv")
all_files.basename = all_files.basename.apply(lambda x: int(x[:6])+400000)
all_files_pre = all_files_pre.loc[(all_files_pre.black_ratio<0.3)]

k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=2050)

train_index_pre, test_index_pre = [], []
for fold, (train_index, test_index) in enumerate(kf.split(all_files_pre)):
    train_index_pre.append(train_index)
    test_index_pre.append(test_index)

#print(parameters)
all_pred = []
all_pred_tr = []
for fold, (train_index, test_index) in enumerate(kf.split(all_files, all_files.Target)):
    X_tr, X_val= all_files.iloc[train_index,:].copy(), all_files.iloc[test_index,:].copy()
    X_tr.to_csv("./temp/final_X_tr%s.csv"%fold, index=False)
    X_val.to_csv("./temp/final_X_val%s.csv"%fold, index=False)
    X_tr_pre, X_val_pre= all_files_pre.iloc[train_index_pre[fold],:].copy(), all_files_pre.iloc[test_index_pre[fold],:].copy()
    X_tr = pd.concat([X_tr, X_tr_pre])
    print( "\nFold ", fold)
    print( "X_tr's size ", X_tr.shape)

In [6]:
fold5182 = np.load("./fusion/test182.npy")
fold5182.shape

(100000, 9)

In [9]:
fold5 = np.squeeze(all_pred).copy()

In [13]:
test_files = pd.read_csv("./dataset/final_test.csv")
ID_values = test_files.loc[:, ["Id_x", "Target"]].values

In [14]:
ID_values[:, 1] = np.argmax(np.mean(np.squeeze(fold5), 0), 1)

AxisError: axis 1 is out of bounds for array of dimension 1

In [19]:
np.concatenate([np.array(fold5)*0.6, fold5182*0.4], 0).shape

(80, 100000, 9)

In [14]:
fold5 = np.load("./results/0706_FL_visitres110_4fold_lr28_03washing_v0_batch256_results.npy")

In [57]:
fold5 = np.load("./results/0702mixed_pre_seresnext5032x4d_110_5fold0_results.npy")

In [10]:
fold5.shape

(40, 100000, 9)

In [56]:
np.mean(fold5, 0)*0.45

array([[0.42776355, 0.31308624, 0.03723255, ..., 0.15021929, 0.18582086,
        0.40452045],
       [0.37840018, 0.23903736, 0.39813834, ..., 0.33415744, 0.07246536,
        0.07959073],
       [0.3605524 , 0.31340173, 0.36258337, ..., 0.32737014, 0.04437453,
        0.05203919],
       ...,
       [0.44522658, 0.3078219 , 0.18811916, ..., 0.06350409, 0.1374954 ,
        0.1379186 ],
       [0.44269735, 0.36306387, 0.16742909, ..., 0.22216186, 0.12501398,
        0.12077025],
       [0.44343403, 0.24962471, 0.06397949, ..., 0.33284593, 0.16351128,
        0.19103773]], dtype=float32)

In [21]:
lgb_result = np.load("./fusion/0712_lgb_fe_test_meanfold_overfit.npy")
lgb_result.shape

(100000, 9)

In [28]:
ID_values[:, 1] = np.argmax(np.mean(np.array([fold5*0.24, result*0.19, fold5182*0.47, lgb_result*0.10]), 0), 1)

In [34]:
ID_values[:, 1] = np.argmax(np.mean(np.array([fold5*0.4, fold5182*0.6]), 0), 1)

In [15]:
ID_values[:, 1] = np.argmax(fold5, 1)

In [16]:
ID_values

array([['./dataset/final_test/000000.jpg', 0],
       ['./dataset/final_test/000001.jpg', 6],
       ['./dataset/final_test/000002.jpg', 0],
       ...,
       ['./dataset/final_test/099997.jpg', 0],
       ['./dataset/final_test/099998.jpg', 0],
       ['./dataset/final_test/099999.jpg', 0]], dtype=object)

In [22]:
with open("./submit/0702_mixed_pre_CE_182vpseresnext5032x4dv0-110+72426vp_5fold_results0519_adjlr_ensemble64_tta.txt", "w") as f:
    for ID, pred in ID_values:
        f.write("%s \t %03d\n"%(os.path.basename(ID)[:-4], pred+1))

In [29]:
with open("./submit/0712_pre_CE_seresnext5032x4dv0-110_5fold+bestacc_meow_senet+CE_182same+lgb_40252015_tta.txt", "w") as f:
    for ID, pred in ID_values:
        f.write("%s \t %03d\n"%(os.path.basename(ID)[:-4], pred+1))

In [16]:
with open("./submit/0717_pre_CE_seresnext5032x4dv0-110_5fold_addcs_tta.txt", "w") as f:
    for ID, pred in ID_values:
        f.write("%s \t %03d\n"%(os.path.basename(ID)[:-4], pred+1))

In [17]:
with open("./submit/0728_pre_CE_seresnext5032x4dv0_5fold_img192_tta.txt", "w") as f:
    for ID, pred in ID_values:
        f.write("%s \t %03d\n"%(os.path.basename(ID)[:-4], pred+1))

In [19]:
a = pd.read_csv("./temp/final_X_tr2.csv")
b = pd.read_csv("./temp/image_X_tr2.csv")

In [13]:
(a == b).all()

Id_x           True
Target         True
black_ratio    True
basename       True
Id_y           True
dtype: bool

In [43]:
dict(b.loc[b.black_ratio<0.3, "Target"].value_counts())

{0: 95942,
 1: 72575,
 2: 40578,
 3: 5264,
 4: 13194,
 5: 49934,
 6: 16861,
 7: 10528,
 8: 14051}

In [22]:
b = pd.read_csv("./temp/final_X_tr2.csv")
b_i = dict(b.loc[b.black_ratio<0.3, "Target"].value_counts())

In [23]:
c = pd.read_csv("./temp/final_X_val2.csv")
c_i = dict(c.loc[c.black_ratio<0.3, "Target"].value_counts())

In [25]:
for i in range(9):
    print("%s/%s"%(b_i[i], c_i[i]), "--", b_i[i]/c_i[i])

96209/23712 -- 4.057397098515519
72473/18256 -- 3.969818141980719
40837/10054 -- 4.061766461110006
5184/1379 -- 3.75924583031182
13161/3271 -- 4.023540201773158
50003/12505 -- 3.998640543782487
16760/4300 -- 3.8976744186046512
10494/2641 -- 3.9734948882998866
13806/3614 -- 3.820143884892086


In [26]:
m_name = "seresnext5032x4d-FLlog-03washing-lr28-v0-final-batch256"

In [27]:
for i in range(5):
    best_model = torch.load("%s/%s_fold_%s_model_best_loss.pth.tar"%(config.best_models, m_name, str(i)))
    torch.save(best_model["state_dict"], "%s_fold_%s_model_best_loss.pth.tar"%( m_name, str(i)))

In [28]:
best_model = torch.load("%s_fold_%s_model_best_loss.pth.tar"%(m_name, str(i)))

In [30]:
best_model.keys()

odict_keys(['module.layer0.conv1.weight', 'module.layer0.bn1.weight', 'module.layer0.bn1.bias', 'module.layer0.bn1.running_mean', 'module.layer0.bn1.running_var', 'module.layer0.bn1.num_batches_tracked', 'module.layer1.0.conv1.weight', 'module.layer1.0.bn1.weight', 'module.layer1.0.bn1.bias', 'module.layer1.0.bn1.running_mean', 'module.layer1.0.bn1.running_var', 'module.layer1.0.bn1.num_batches_tracked', 'module.layer1.0.conv2.weight', 'module.layer1.0.bn2.weight', 'module.layer1.0.bn2.bias', 'module.layer1.0.bn2.running_mean', 'module.layer1.0.bn2.running_var', 'module.layer1.0.bn2.num_batches_tracked', 'module.layer1.0.conv3.weight', 'module.layer1.0.bn3.weight', 'module.layer1.0.bn3.bias', 'module.layer1.0.bn3.running_mean', 'module.layer1.0.bn3.running_var', 'module.layer1.0.bn3.num_batches_tracked', 'module.layer1.0.se_module.fc1.weight', 'module.layer1.0.se_module.fc1.bias', 'module.layer1.0.se_module.fc2.weight', 'module.layer1.0.se_module.fc2.bias', 'module.layer1.0.downsample.

In [7]:
#!/usr/bin/env python
# -*- coding:utf-8 -*-
# Author: Xiangtai Li(lxtpku@pku.edu.cn)
# Pytorch Implementation of Octave Conv Operation
# This version use nn.Conv2d because alpha_in always equals alpha_out

import torch
import torch.nn as nn

In [3]:
class OctaveConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, alpha=0.5, stride=1, padding=1, dilation=1,
                 groups=1, bias=False):
        super(OctaveConv, self).__init__()
        kernel_size = kernel_size[0]
        self.h2g_pool = nn.AvgPool2d(kernel_size=(2, 2), stride=2)
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='nearest')
        self.stride = stride
        self.l2l = torch.nn.Conv2d(int(alpha * in_channels), int(alpha * out_channels),
                                   kernel_size, 1, padding, dilation, groups, bias)
        self.l2h = torch.nn.Conv2d(int(alpha * in_channels), out_channels - int(alpha * out_channels),
                                   kernel_size, 1, padding, dilation, groups, bias)
        self.h2l = torch.nn.Conv2d(in_channels - int(alpha * in_channels), int(alpha * out_channels),
                                   kernel_size, 1, padding, dilation, groups, bias)
        self.h2h = torch.nn.Conv2d(in_channels - int(alpha * in_channels),
                                   out_channels - int(alpha * out_channels),
                                   kernel_size, 1, padding, dilation, groups, bias)

    def forward(self, x):
        X_h, X_l = x

        if self.stride ==2:
            X_h, X_l = self.h2g_pool(X_h), self.h2g_pool(X_l)

        X_h2l = self.h2g_pool(X_h)

        X_h2h = self.h2h(X_h)
        X_l2h = self.l2h(X_l)

        X_l2l = self.l2l(X_l)
        X_h2l = self.h2l(X_h2l)
        
        X_l2h = self.upsample(X_l2h)
        X_h = X_l2h + X_h2h
        X_l = X_h2l + X_l2l

        return X_h, X_l


class FirstOctaveConv(nn.Module):
    def __init__(self, in_channels, out_channels,kernel_size, alpha=0.5, stride=1, padding=1, dilation=1,
                 groups=1, bias=False):
        super(FirstOctaveConv, self).__init__()
        self.stride = stride
        kernel_size = kernel_size[0]
        self.h2g_pool = nn.AvgPool2d(kernel_size=(2, 2), stride=2)
        self.h2l = torch.nn.Conv2d(in_channels, int(alpha * out_channels),
                                   kernel_size, 1, padding, dilation, groups, bias)
        self.h2h = torch.nn.Conv2d(in_channels, out_channels - int(alpha * out_channels),
                                   kernel_size, 1, padding, dilation, groups, bias)

    def forward(self, x):
        if self.stride ==2:
            x = self.h2g_pool(x)

        X_h2l = self.h2g_pool(x)
        X_h = x
        X_h = self.h2h(X_h)
        X_l = self.h2l(X_h2l)

        return X_h, X_l


class LastOctaveConv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, alpha=0.5, stride=1, padding=1, dilation=1,
                 groups=1, bias=False):
        super(LastOctaveConv, self).__init__()
        self.stride = stride
        kernel_size = kernel_size[0]
        self.h2g_pool = nn.AvgPool2d(kernel_size=(2,2), stride=2)

        self.l2h = torch.nn.Conv2d(int(alpha * in_channels), out_channels,
                                   kernel_size, 1, padding, dilation, groups, bias)
        self.h2h = torch.nn.Conv2d(in_channels - int(alpha * in_channels),
                                   out_channels,
                                   kernel_size, 1, padding, dilation, groups, bias)
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='nearest')

    def forward(self, x):
        X_h, X_l = x

        if self.stride ==2:
            X_h, X_l = self.h2g_pool(X_h), self.h2g_pool(X_l)

        X_l2h = self.l2h(X_l)
        X_h2h = self.h2h(X_h)
        X_l2h = self.upsample(X_l2h)
        
        X_h = X_h2h + X_l2h

        return X_h


class OctaveCBR(nn.Module):
    def __init__(self,in_channels, out_channels, kernel_size=(3,3),alpha=0.5, stride=1, padding=1, dilation=1,
                 groups=1, bias=False, norm_layer=nn.BatchNorm2d):
        super(OctaveCBR, self).__init__()
        self.conv = OctaveConv(in_channels,out_channels,kernel_size, alpha, stride, padding, dilation, groups, bias)
        self.bn_h = norm_layer(int(out_channels*(1-alpha)))
        self.bn_l = norm_layer(int(out_channels*alpha))
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x_h, x_l = self.conv(x)
        x_h = self.relu(self.bn_h(x_h))
        x_l = self.relu(self.bn_l(x_l))
        return x_h, x_l


class OctaveCB(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(3,3), alpha=0.5, stride=1, padding=1, dilation=1,
                 groups=1, bias=False, norm_layer=nn.BatchNorm2d):
        super(OctaveCB, self).__init__()
        self.conv = OctaveConv(in_channels, out_channels, kernel_size, alpha, stride, padding, dilation,
                               groups, bias)
        self.bn_h = norm_layer(int(out_channels * (1 - alpha)))
        self.bn_l = norm_layer(int(out_channels * alpha))

    def forward(self, x):
        x_h, x_l = self.conv(x)
        x_h = self.bn_h(x_h)
        x_l = self.bn_l(x_l)
        return x_h, x_l


class FirstOctaveCBR(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(3,3),alpha=0.5, stride=1, padding=1, dilation=1,
                 groups=1, bias=False,norm_layer=nn.BatchNorm2d):
        super(FirstOctaveCBR, self).__init__()
        self.conv = FirstOctaveConv(in_channels,out_channels,kernel_size, alpha,stride,padding,dilation,groups,bias)
        self.bn_h = norm_layer(int(out_channels * (1 - alpha)))
        self.bn_l = norm_layer(int(out_channels * alpha))
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x_h, x_l = self.conv(x)
        x_h = self.relu(self.bn_h(x_h))
        x_l = self.relu(self.bn_l(x_l))
        return x_h, x_l


class LastOCtaveCBR(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(3,3), alpha=0.5, stride=1, padding=1, dilation=1,
                 groups=1, bias=False, norm_layer=nn.BatchNorm2d):
        super(LastOCtaveCBR, self).__init__()
        self.conv = LastOctaveConv(in_channels, out_channels, kernel_size, alpha, stride, padding, dilation, groups, bias)
        self.bn_h = norm_layer(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x_h = self.conv(x)
        x_h = self.relu(self.bn_h(x_h))
        return x_h


class FirstOctaveCB(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=(3,3), alpha=0.5,stride=1, padding=1, dilation=1,
                 groups=1, bias=False, norm_layer=nn.BatchNorm2d):
        super(FirstOctaveCB, self).__init__()
        self.conv = FirstOctaveConv(in_channels,out_channels,kernel_size, alpha,stride,padding,dilation,groups,bias)
        self.bn_h = norm_layer(int(out_channels * (1 - alpha)))
        self.bn_l = norm_layer(int(out_channels * alpha))
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x_h, x_l = self.conv(x)
        x_h = self.bn_h(x_h)
        x_l = self.bn_l(x_l)
        return x_h, x_l


class LastOCtaveCB(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, alpha=0.5, stride=1, padding=1, dilation=1,
                 groups=1, bias=False, norm_layer=nn.BatchNorm2d):
        super(LastOCtaveCB, self).__init__()
        self.conv = LastOctaveConv( in_channels, out_channels, kernel_size, alpha, stride, padding, dilation, groups, bias)
        self.bn_h = norm_layer(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x_h = self.conv(x)
        x_h = self.bn_h(x_h)
        return x_h

In [7]:
class OctaveConv_1(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, alpha_in=0.5, alpha_out=0.5, stride=1, padding=0, dilation=1,
                 groups=1, bias=False):
        super(OctaveConv, self).__init__()
        self.downsample = nn.AvgPool2d(kernel_size=(2, 2), stride=2)
        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        assert stride == 1 or stride == 2, "Stride should be 1 or 2."
        self.stride = stride
        assert 0 <= alpha_in <= 1 and 0 <= alpha_out <= 1, "Alphas should be in the interval from 0 to 1."
        self.alpha_in, self.alpha_out = alpha_in, alpha_out
        self.conv_l2l = None if alpha_in == 0 or alpha_out == 0 else \
                        nn.Conv2d(int(alpha_in * in_channels), int(alpha_out * out_channels),
                                  kernel_size, 1, padding, dilation, groups, bias)
        self.conv_l2h = None if alpha_in == 0 or alpha_out == 1 else \
                        nn.Conv2d(int(alpha_in * in_channels), out_channels - int(alpha_out * out_channels),
                                  kernel_size, 1, padding, dilation, groups, bias)
        self.conv_h2l = None if alpha_in == 1 or alpha_out == 0 else \
                        nn.Conv2d(in_channels - int(alpha_in * in_channels), int(alpha_out * out_channels),
                                  kernel_size, 1, padding, dilation, groups, bias)
        self.conv_h2h = None if alpha_in == 1 or alpha_out == 1 else \
                        nn.Conv2d(in_channels - int(alpha_in * in_channels), out_channels - int(alpha_out * out_channels),
                                  kernel_size, 1, padding, dilation, groups, bias)

    def forward(self, x):
        x_h, x_l = x if type(x) is tuple else (x, None)

        if x_h is not None:
            x_h = self.downsample(x_h) if self.stride == 2 else x_h
            x_h2h = self.conv_h2h(x_h)
            x_h2l = self.conv_h2l(self.downsample(x_h)) if self.alpha_out > 0 else None
        if x_l is not None:
            x_l2h = self.conv_l2h(x_l)
            x_l2h = self.upsample(x_l2h) if self.stride == 1 else x_l2h
            x_l2l = self.downsample(x_l) if self.stride == 2 else x_l
            x_l2l = self.conv_l2l(x_l2l) if self.alpha_out > 0 else None 
            x_h = x_l2h + x_h2h
            x_l = x_h2l + x_l2l if x_h2l is not None and x_l2l is not None else None
            return x_h, x_l
        else:
            return x_h2h, x_h2l

In [9]:
FOCconv = OctaveConv_1(kernel_size=3, in_channels=3, out_channels=128).cuda()
x_out, y_out = FOCconv(i)
print("First: ", x_out.size(), y_out.size())

TypeError: super(type, obj): obj must be an instance or subtype of type

In [5]:
if __name__ == '__main__':
    # nn.Conv2d
    high = torch.Tensor(1, 64, 32, 32).cuda()
    low = torch.Tensor(1, 192, 16, 16).cuda()
    # test Oc conv
    OCconv = OctaveConv(kernel_size=(3,3),in_channels=256,out_channels=512,bias=False,stride=2,alpha=0.75).cuda()
    i = high,low
    x_out,y_out = OCconv(i)
    print(x_out.size())
    print(y_out.size())

    i = torch.Tensor(1, 3, 24, 182).cuda()
    FOCconv = FirstOctaveConv(kernel_size=(3, 3), in_channels=3, out_channels=128).cuda()
    x_out, y_out = FOCconv(i)
    print("First: ", x_out.size(), y_out.size())
    # test last Octave Cov
    LOCconv = LastOctaveConv(kernel_size=(3, 3), in_channels=256, out_channels=128, alpha=0.75).cuda()
    i = high, low
    out = LOCconv(i)
    print("Last: ", out.size())

    # test OCB
    ocb = OctaveCB(in_channels=256, out_channels=128, alpha=0.75).cuda()
    i = high, low
    x_out_h, y_out_l = ocb(i)
    print("OCB:",x_out_h.size(),y_out_l.size())

    # test last OCB
    ocb_last = LastOCtaveCBR(256, 128, alpha=0.75).cuda()
    i = high, low
    x_out_h = ocb_last(i)
    print("Last OCB", x_out_h.size())

torch.Size([1, 128, 16, 16])
torch.Size([1, 384, 8, 8])
First:  torch.Size([1, 64, 24, 182]) torch.Size([1, 64, 12, 91])
Last:  torch.Size([1, 128, 32, 32])
OCB: torch.Size([1, 32, 32, 32]) torch.Size([1, 96, 16, 16])
Last OCB torch.Size([1, 128, 32, 32])


D:\Softwares\Anaconda3\envs\test\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


In [20]:
from models.oct_resnet import oct_resnet110, oct_resnet50
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from torchsummary import summary

In [22]:
from models.resnet4cifar import resnet110

In [23]:
summary(resnet110().cuda(), (3, 224, 224))

E:\DL\LiZeda\InterestingProjects\competition\bd\models\resnet4cifar.py:38: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 16, 224, 224]             432
       BatchNorm2d-2         [-1, 16, 224, 224]              32
              ReLU-3         [-1, 16, 224, 224]               0
            Conv2d-4         [-1, 16, 224, 224]           2,304
       BatchNorm2d-5         [-1, 16, 224, 224]              32
            Conv2d-6         [-1, 16, 224, 224]           2,304
       BatchNorm2d-7         [-1, 16, 224, 224]              32
        BasicBlock-8         [-1, 16, 224, 224]               0
            Conv2d-9         [-1, 16, 224, 224]           2,304
      BatchNorm2d-10         [-1, 16, 224, 224]              32
           Conv2d-11         [-1, 16, 224, 224]           2,304
      BatchNorm2d-12         [-1, 16, 224, 224]              32
       BasicBlock-13         [-1, 16, 224, 224]               0
           Conv2d-14         [-1, 16, 2

In [18]:
summary(oct_resnet110().cuda(), (3, 224, 224))

D:\Softwares\Anaconda3\envs\test\lib\site-packages\torch\nn\modules\upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")


AttributeError: 'tuple' object has no attribute 'size'

In [24]:
from pretrainedmodels import models

In [42]:
from collections import OrderedDict

Sequential(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)

In [45]:
layer0_modules = [
('conv1', nn.Conv2d(3, 64, 3, stride=2, padding=1,
                    bias=False)),
('bn1', nn.BatchNorm2d(64)),
('relu1', nn.ReLU(inplace=True)),
('conv2', nn.Conv2d(64, 64, 3, stride=1, padding=1,
                    bias=False)),
('bn2', nn.BatchNorm2d(64)),
('relu2', nn.ReLU(inplace=True)),
('conv3', nn.Conv2d(64, 64, 3, stride=1, padding=1,
                    bias=False)),
]

nn.Sequential(OrderedDict(layer0_modules))

Sequential(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)

In [38]:
se50 = models.se_resnext50_32x4d(input_3x3=True)
se50.layer0.conv1 = nn.Conv2d(7, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
se50.avg_pool = nn.AdaptiveAvgPool2d(1)

RuntimeError: Error(s) in loading state_dict for SENet:
	Missing key(s) in state_dict: "layer0.conv2.weight", "layer0.bn2.running_mean", "layer0.bn2.weight", "layer0.bn2.bias", "layer0.bn2.running_var", "layer0.conv3.weight", "layer0.bn3.running_mean", "layer0.bn3.weight", "layer0.bn3.bias", "layer0.bn3.running_var". 
	size mismatch for layer0.conv1.weight: copying a param of torch.Size([64, 3, 3, 3]) from checkpoint, where the shape is torch.Size([64, 3, 7, 7]) in current model.

In [37]:
summary(se50.cuda(), (7, 24, 26))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 12, 13]           4,032
       BatchNorm2d-2           [-1, 64, 12, 13]             128
              ReLU-3           [-1, 64, 12, 13]               0
            Conv2d-4           [-1, 64, 12, 13]          36,864
       BatchNorm2d-5           [-1, 64, 12, 13]             128
              ReLU-6           [-1, 64, 12, 13]               0
            Conv2d-7           [-1, 64, 12, 13]          36,864
       BatchNorm2d-8           [-1, 64, 12, 13]             128
              ReLU-9           [-1, 64, 12, 13]               0
        MaxPool2d-10             [-1, 64, 6, 6]               0
           Conv2d-11            [-1, 128, 6, 6]           8,192
      BatchNorm2d-12            [-1, 128, 6, 6]             256
             ReLU-13            [-1, 128, 6, 6]               0
           Conv2d-14            [-1, 12

In [47]:
import numpy as np

In [50]:
X = np.arange(10).reshape(2, 5)

In [51]:
X

array([[0, 1, 2, 3, 4],
       [5, 6, 7, 8, 9]])

In [53]:
X / np.sum(X, 0, keepdims=True)

array([[0.        , 0.14285714, 0.22222222, 0.27272727, 0.30769231],
       [1.        , 0.85714286, 0.77777778, 0.72727273, 0.69230769]])